In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/MiniProject/Project/GA

/content/drive/.shortcut-targets-by-id/1R5lktF0Aag9Aa-GqEI_uQJTKOkHYAbQG/MiniProject/Project/GA


In [3]:
import numpy as np
import copy
import random 
import os
import time

In [4]:
def loadData(path):
    global N, m, M, d, s, e, date
    f = open(path, "r")

    param = f.readline().split()
    N = int(param[0])
    m = int(param[1])
    M = int(param[2])

    d = []
    s = []
    e = []
    for i in range(N):
        line = f.readline().split()
        d.append(int(line[0]))
        s.append(int(line[1]))
        e.append(int(line[2]))

    f.close()

    date = np.max(e)+1

In [5]:
def cost(sol):
    amount = np.zeros(date)
    violation = 0
    for i in range(N):
        amount[sol[i]] += int(d[i])
    for i in amount:
        if (i > M):
            violation += i - M
        elif (i > 0 and i < m):
            violation += min(m-i, i)
    different = np.max(amount) - np.min(amount)
    return int(different + 1000 * violation)

In [6]:
def initial_solution(SEED):
    random.seed(SEED)
    sol = [0]*N
    amount = np.zeros(date)
    global flag
    flag = [0]*date

    # cánh đồng nào thu nhiều thì ưu tiên chọn trước
    order = np.argsort(-np.array(d))
    for i in order:
        # sol[i] = random.randint(s[i],e[i])
        temp1 = [j for j in range(s[i], e[i]+1) if flag[j] <= 1]
        temp2 = [j for j in range(s[i], e[i]+1)
                 if (flag[j] == 2 and amount[j] + d[i] <= M)]
        if (len(temp1) != 0):
            day = random.choice(temp1)
        elif (len(temp2) != 0):
            day = random.choice(temp2)
        else:
            day = random.randint(s[i], e[i])

        sol[i] = day
        amount[day] += d[i]
        if (amount[day] > M):
            flag[day] = 3  # đã đến giới hạn
        elif (amount[day] >= m):
            flag[day] = 2
        elif (amount[day] > 0):
            flag[day] = 1

    # sol = jump(sol,SEED)
    # print(amount)
    return sol

In [7]:
def sort(sol):
  return sol[-1]

In [8]:
def GA(SEED):
    random.seed(SEED)
    # khởi tạo quần thể
    population = []
    solution_gen = []
    for i in range(300):
        sol = [0]*N
        sol = initial_solution(SEED + i*10)
        sol.append(cost(sol))
        population.append(sol)
    population.sort(reverse=False, key=sort)  # sắp xếp cá thể theo cost

    for k in range(300):
        # while(1):
        lamda = random.uniform(0, 1)  # tỉ lệ lai ghép và đột biến
        if (lamda < 0.9):
            # lai ghép
            new_generation = []
            size = int(len(population)/10)
            for _ in range(int((len(population) - size)/2)):
                # chọn cá thể lai ghép đầu tiên
                number1 = random.randint(0, int(len(population)/2))
                # chọn cá thể lai ghép thư hai
                number2 = random.randint(0, int(len(population)/2))
                while(number1 == number2):
                    number2 = random.randint(0, int(len(population)/2))
                x1 = random.randint(1, N-1)  # chọn điểm cắt lai ghép thứ 1
                x2 = random.randint(1, N-1)  # chọn điểm cắt lai ghép thứ 2

                while(x1 == x2):
                    x1 = random.randint(1, N-1)
                if (x1 > x2):
                    x1, x2 = x2, x1

                new_individual_1 = [0]*N  # cá thể con 1
                new_individual_2 = [0]*N  # cá thể con 2
                for i in range(x1):
                    new_individual_1[i] = population[number1][i]
                    new_individual_2[i] = population[number2][i]
                for i in range(x1, x2):
                    new_individual_1[i] = population[number2][i]
                    new_individual_2[i] = population[number1][i]
                for i in range(x2, N):
                    new_individual_1[i] = population[number1][i]
                    new_individual_2[i] = population[number2][i]

                new_individual_1.append(cost(new_individual_1))
                new_generation.append(new_individual_1)
                new_individual_2.append(cost(new_individual_2))
                new_generation.append(new_individual_2)

            # del population[size:]
            # for i in new_generation:
            #   population.append(i)
            for i in population[:size]:
                new_generation.append(i)
            population = new_generation
            # sắp xếp cá thể theo cost
            population.sort(reverse=False, key=sort)
        else:
            # đột biến
            # chọn cá thể đột biến
            number3 = random.randint(0, len(population)-1)
            x = random.randint(0, N-1)  # chọn gen đột biến
            while(s[x] == e[x]):
                x = random.randint(0, N-1)

            new_individual_3 = [0]*N  # khởi tạo cá thể đột biến
            for i in range(N):
                new_individual_3[i] = population[number3][i]

            new_individual_3[x] = random.randint(s[x], e[x])
            while(new_individual_3[x] == population[number3][x]):
                new_individual_3[x] = random.randint(s[x], e[x])
            new_individual_3 = np.append(
                new_individual_3, cost(new_individual_3))
            del population[-1]  # xóa cá thể kém
            # thêm cá thể đột biến vào quần thể
            population.append(new_individual_3)
            # sắp xếp cá thể theo cost
            population.sort(reverse=False, key=sort)
        solution_gen.append(population[0])
        if (population[0][-1] < 50):
            break
    return solution_gen

In [9]:
def logOutput(output_path, Name, time, SEED, solution_gen):
    sol = solution_gen[-1]
    direct = output_path + ".txt"

    with open(direct, "w+") as f:
      f.write("Name: "+Name+"\n")
      f.write("Time: {} \n".format(time))
      f.write("Result: {}\n".format(sol[-1]))
      f.write("Solution: ")
      for i in range(len(sol) - 1):
          f.write(" {}".format(sol[i]))
      f.write("\n")
      
      amount = np.zeros(date)
      day = [[] for i in range(date)]
      for i in range(N):
          amount[sol[i]] += d[i]
          day[sol[i]].append(i)

      for i in range(date):
          f.write("{} : {} ({})\n".format(i, day[i], int(amount[i])))

In [10]:
def logOutput2(output_path, Name, SEED, solution_gen):
    sol = solution_gen[-1]
    direct = output_path + "_gen.txt"

    with open(direct, "w+") as f:
      f.write("File name: " + Name + "\n")
      f.write("Result: " + str(sol[-1]) + "\n")
      for i in range(1, len(solution_gen)+1):
          f.write("Gen " + str(i) + ":" + str(solution_gen[i-1]) + "\n")

In [11]:
for i in range(1, 5):
    os.mkdir("Output/Type"+str(i)+"Small")
    os.mkdir("Output/Type"+str(i)+"Big")

**MAIN**

In [12]:
for i in range(1, 5):
  Type = f"Type{i}Big"
  src = f"Data/{Type}"
  for file in os.listdir(src):
      Name = file.split('.')[0]
      loadData(src + "/" + file)
      for SEED in range(10):
          outputPath = "Output/" + Type + "/" + Name+"_SEED_"+str(SEED)
          print(outputPath)
          
          start = time.time()
          solution_gen = GA(SEED)
          run_time = time.time() - start

          logOutput(outputPath, Name, run_time, SEED, solution_gen)
          logOutput2(outputPath, Name, SEED, solution_gen)

Output/Type1Big/200-20-30_SEED_0
Output/Type1Big/200-20-30_SEED_1
Output/Type1Big/200-20-30_SEED_2
Output/Type1Big/200-20-30_SEED_3
Output/Type1Big/200-20-30_SEED_4
Output/Type1Big/200-20-30_SEED_5
Output/Type1Big/200-20-30_SEED_6
Output/Type1Big/200-20-30_SEED_7
Output/Type1Big/200-20-30_SEED_8
Output/Type1Big/200-20-30_SEED_9
Output/Type1Big/200-30-45_SEED_0
Output/Type1Big/200-30-45_SEED_1
Output/Type1Big/200-30-45_SEED_2
Output/Type1Big/200-30-45_SEED_3
Output/Type1Big/200-30-45_SEED_4
Output/Type1Big/200-30-45_SEED_5
Output/Type1Big/200-30-45_SEED_6
Output/Type1Big/200-30-45_SEED_7
Output/Type1Big/200-30-45_SEED_8
Output/Type1Big/200-30-45_SEED_9
Output/Type1Big/200-40-60_SEED_0
Output/Type1Big/200-40-60_SEED_1
Output/Type1Big/200-40-60_SEED_2
Output/Type1Big/200-40-60_SEED_3
Output/Type1Big/200-40-60_SEED_4
Output/Type1Big/200-40-60_SEED_5
Output/Type1Big/200-40-60_SEED_6
Output/Type1Big/200-40-60_SEED_7
Output/Type1Big/200-40-60_SEED_8
Output/Type1Big/200-40-60_SEED_9
Output/Typ